### Convolutionan NN using cifar 10
Following literature from [cs231n](http://cs231n.github.io/convolutional-networks/)

In [1]:
#!pip install h5py

In [2]:
import numpy as np 
np.random.seed(1337) 

In [4]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential, Model #model for the functional
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, Input

In [5]:
batch_size = 32
num_classes = 10
epochs = 2

In [6]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
model = Sequential()

model.add(Conv2D(16, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(Activation('relu'))

model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu')) #rectuied linear unit; 0 for x<0, identity for x>0

model.add(Flatten()) #reduce one dimension

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 8)         1160      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                72010     
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total para

In [11]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

### Saving model weights 

In [13]:
import h5py  # to ensure we have this package installed 

from keras.callbacks import ModelCheckpoint
#epoch number, 2 decimals - valaccuracy 2 float
filepath="checkpoints/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1, 
                             mode='max')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          callbacks=[checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 45s 909us/step - loss: 1.5486 - acc: 0.4591 - val_loss: 1.4840 - val_acc: 0.4798

Epoch 00001: saving model to checkpoints/weights-improvement-01-0.48.hdf5
Epoch 2/2
50000/50000 [==============================] - 40s 790us/step - loss: 1.4418 - acc: 0.4938 - val_loss: 1.4248 - val_acc: 0.4935

Epoch 00002: saving model to checkpoints/weights-improvement-02-0.49.hdf5


In [14]:
#model.fit(x_train, y_train,
#          batch_size=batch_size,
#          epochs=epochs,
#          validation_data=(x_test, y_test))
#without the checkpoint h5 files

In [15]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 3s 306us/step


[1.4247742393493652, 0.4935]

### Loading Saved Model 

In [17]:
# just create the model as before and then load weight 
model.load_weights("checkpoints/weights-improvement-01-0.48.h5")   

In [18]:
model.evaluate(x_test, y_test)   # produces the same results 

10000/10000 [==============================] - 3s 312us/step


[1.484040534210205, 0.4798]

---

Here we shoukd try 100 epochs on a GPU machine, not on local anymore

Let's try a new configuration with **Pooling layers**

In [14]:
from keras.layers import Dropout, MaxPooling2D

In [15]:
model = Sequential()

model.add(Conv2D(16, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(Activation('relu'))

model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2))) #reduce the amount of parameters and computation in the network
model.add(Dropout(0.25))    # new layer 

model.add(Flatten())

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
activation_10 (Activation)   (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 8)         1160      
_________________________________________________________________
activation_11 (Activation)   (None, 30, 30, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1800)              0         
__________

Now go ahead and look at the Keras example for CIFAR10: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py 

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

### Inception Module

Take a look at the inception module figure here: https://arxiv.org/pdf/1409.4842.pdf

In [23]:
# input layer is the same as our typical CNN model 
inputs = Input(shape=(32, 32, 3))

#the towers are the different blocks were a layer can split it's work, then at the final the concatenation filter will integrate them

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu', name='t1_conv')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs) #strides (jumps)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)


x = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1) 

# Rest of the model, again, remains the same 

x = Conv2D(8, (3, 3))(x)    
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x) 



In [24]:
model = Model([inputs], output)

In [25]:
model.summary()  # Notice the 'Connected To' in the summary 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 32, 32, 64)   256         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 32, 32, 64)   256         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 32, 32, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
t1_conv (C

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


#model.fit(x_train, y_train,
#          batch_size=batch_size,
#          epochs=epochs,
#          validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
 1952/50000 [>.............................] - ETA: 16:47 - loss: 2.2656 - acc: 0.1547